In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf  # Add this line to import TensorFlow
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# Load the preprocessed data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Set up image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='dataset/preprocessed/train',
    x_col='Name',
    y_col='HeadCount',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    directory='dataset/preprocessed/train',
    x_col='Name',
    y_col='HeadCount',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

Found 8186 validated image filenames.
Found 2047 validated image filenames.


In [3]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load the pre-trained MobileNetV2 model without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model for regression
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(1, activation='linear')(x)  # Single output for head count

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [4]:
# Calculate the steps_per_epoch and validation_steps
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

print("Steps per epoch:", steps_per_epoch)
print("Validation steps:", validation_steps)

Steps per epoch: 256
Validation steps: 64


In [5]:
# Set the number of epochs and steps per epoch for training
epochs = 10
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps
)

# Save the trained model
model.save('head_count_model.h5')

/var/folders/qv/7r3vg1ns6qbgvs0t3_dxc3qm0000gn/T/ipykernel_2786/1932027379.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10


2024-03-29 22:10:07.062836: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-03-29 22:10:07.076913: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


256/256 [==============================] - ETA: 0s - loss: 11.4701 - mae: 2.3021

2024-03-29 22:12:45.699886: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


256/256 [==============================] - 196s 753ms/step - loss: 11.4701 - mae: 2.3021 - val_loss: 9.6624 - val_mae: 2.0791
Epoch 2/10
256/256 [==============================] - 194s 757ms/step - loss: 9.4518 - mae: 2.0742 - val_loss: 7.7003 - val_mae: 1.9451
Epoch 3/10
256/256 [==============================] - 186s 724ms/step - loss: 8.9131 - mae: 1.9981 - val_loss: 7.4352 - val_mae: 1.9308
Epoch 4/10
256/256 [==============================] - 187s 728ms/step - loss: 8.3037 - mae: 1.9427 - val_loss: 7.3204 - val_mae: 1.8755
Epoch 5/10
256/256 [==============================] - 187s 729ms/step - loss: 8.0859 - mae: 1.9217 - val_loss: 8.3509 - val_mae: 1.9561
Epoch 6/10
256/256 [==============================] - 220s 858ms/step - loss: 8.0993 - mae: 1.9384 - val_loss: 7.4892 - val_mae: 1.8489
Epoch 7/10
256/256 [==============================] - 218s 852ms/step - loss: 7.8113 - mae: 1.8995 - val_loss: 7.2035 - val_mae: 1.8121
Epoch 8/10
256/256 [==============================] - 223s

In [6]:
from tensorflow.keras.models import load_model

loaded_model = load_model('head_count_model.h5')

In [7]:
# Evaluate the model on the training data
train_loss, train_rmse = model.evaluate(train_generator)

print(f"Training Loss: {train_loss}")
print(f"Training RMSE: {train_rmse}")

2024-03-29 22:44:28.316143: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


256/256 [==============================] - 170s 664ms/step - loss: 7.0376 - mae: 1.8576
Training Loss: 7.0375847816467285
Training RMSE: 1.8575537204742432


In [8]:
test_df = pd.read_csv('test.csv')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='dataset/preprocessed/test',
    x_col='Name',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False 
)

Found 3963 validated image filenames.


In [9]:
predictions = model.predict(test_generator)

2024-03-29 22:47:44.612746: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


124/124 [==============================] - 83s 662ms/step


In [10]:
import numpy as np
import pandas as pd

# Make predictions on the test set
predictions = model.predict_generator(test_generator, steps=len(test_generator))

# Create a DataFrame for the submission with float predictions
submission_df = pd.DataFrame({
    'Name': test_df['Name'],
    'HeadCount': predictions.flatten()
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_main7.csv', index=False)

/var/folders/qv/7r3vg1ns6qbgvs0t3_dxc3qm0000gn/T/ipykernel_2786/1634233759.py:5: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(test_generator, steps=len(test_generator))
2024-03-29 22:49:49.765088: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
